In [1]:
from utils.my_llm_utils import *
import pandas as pd

from utils.crawler_utils import crawl_url
from utils.my_llm_utils import summarize_text
from utils.tavily_search_utils import search_tavily



In [ ]:
# load the LLM
llm = get_llm()

# Check which llm has been loaded
if hasattr(llm, 'model'):
	print(llm.model)  # check loaded model
else:
	print(llm)

In [ ]:
# test queries

# # test query
# response = llm.invoke("What is the capital of Russia?")
# print(response)

testUrl = "https://www.crn.com/news/ai/2024/accenture-to-train-30-000-staff-on-nvidia-ai-tech-in-blockbuster-deal"



In [ ]:
# ------------------------------------------------------
# test search_tavily
# ------------------------------------------------------
search_tavily("NVIDIA AI")

#------------------------------------------------------
# test crawl_url
# ------------------------------------------------------
text = crawl_url(testUrl)


# ------------------------------------------------------
# test summarize_text
# ------------------------------------------------------
summary = summarize_text(text)
print(summary)


In [4]:
# Config
INPUT_FILE = 'data/AI Partnerships sample.csv'
OUTPUT_FILE = 'data/AI_Partnerships_Updated.csv'

# Load Data
df = pd.read_csv(INPUT_FILE)

In [ ]:
query = "Accenture Nvidia partnership"
search_response = search_tavily(query)
print(search_response)

text = search_response['results'][0]['content']

summary = summarize_text(text)
print(summary)


{'query': 'Accenture Nvidia partnership', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'ServiceNow, NVIDIA, and Accenture Team to Accelerate Generative AI ...', 'url': 'https://newsroom.accenture.com/news/2023/servicenow-nvidia-and-accenture-team-to-accelerate-generative-ai-adoption-for-enterprises', 'content': 'ServiceNow, NVIDIA, and Accenture Team to Accelerate Generative AI Adoption for Enterprises July 26, 2023 ServiceNow, NVIDIA, and Accenture Team to Accelerate Generative AI Adoption for Enterprises First-of-its-kind AI Lighthouse program helps customers create generative AI-based initiatives to transform their business operations SANTA CLARA, Calif.; July 26, 2023 – ServiceNow (NYSE: NOW), NVIDIA (NASDAQ: NVDA), and Accenture (NYSE: ACN) today announced the launch of AI Lighthouse, a first-of-its-kind program designed to fast-track the development and adoption of enterprise generative AI capabilities. Expanding on existing strategic partnersh